In [2]:
from transformers import BertForMaskedLM
import pandas as pd
from geneformer.tokenizer import (
    GENE_MEDIAN_FILE, 
    TOKEN_DICTIONARY_FILE,
    tokenize_cell
)
import pickle
import glob
import numpy as np
from datasets import Dataset

class GenericTranscriptomeTokenizer:
    def __init__(
        self,
        custom_attr_name_dict=None,
        nproc=1,
        gene_median_file=GENE_MEDIAN_FILE,
        token_dictionary_file=TOKEN_DICTIONARY_FILE,
    ):
        """
        Initialize tokenizer.
        Parameters
        ----------
        custom_attr_name_dict : dict
            Dictionary of custom attributes to be added to the dataset.
            Keys are the names of the attributes in the loom file.
            Values are the names of the attributes in the dataset.
        nproc : int
            Number of processes to use for dataset mapping.
        gene_median_file : Path
            Path to pickle file containing dictionary of non-zero median
            gene expression values across Genecorpus-30M.
        token_dictionary_file : Path
            Path to pickle file containing token dictionary (Ensembl IDs:token).
        """
        # dictionary of custom attributes {output dataset column name: input .loom column name}            
        self.custom_attr_name_dict = custom_attr_name_dict if custom_attr_name_dict is not None else {}

        # number of processes for dataset mapping
        self.nproc = nproc

        # load dictionary of gene normalization factors
        # (non-zero median value of expression across Genecorpus-30M)
        with open(gene_median_file, "rb") as f:
            self.gene_median_dict = pickle.load(f)

        # load token dictionary (Ensembl IDs:token)
        with open(token_dictionary_file, "rb") as f:
            self.gene_token_dict = pickle.load(f)

        # gene keys for full vocabulary
        self.gene_keys = list(self.gene_median_dict.keys())

        # protein-coding and miRNA gene list dictionary for selecting .loom rows for tokenization
        self.genelist_dict = dict(zip(self.gene_keys, [True] * len(self.gene_keys)))
        
        

    def tokenize_data(self, data_directory, file_type, output_directory=None, output_prefix="tokenized_txs"):
        """
        Tokenize .loom files in loom_data_directory and save as tokenized .dataset in output_directory.
        Parameters
        ----------
        loom_data_directory : Path
            Path to directory containing loom files
        output_directory : Path
            Path to directory where tokenized data will be saved as .dataset
        output_prefix : str
            Prefix for output .dataset
        """
        tokenized_cells, cell_metadata = self.tokenize_files(data_directory, file_type)
        tokenized_dataset = self.create_dataset(tokenized_cells, cell_metadata)
        
        if output_directory is not None:
            output_path = (Path(output_directory) / output_prefix).with_suffix(".dataset")
            tokenized_dataset.save_to_disk(output_path)
        return tokenized_dataset

    def tokenize_files(self, data_directory, file_type):
        tokenized_cells = []
        cell_metadata = {attr_key: [] for attr_key in self.custom_attr_name_dict.keys()}
        
        if file_type in ["csv", "tsv"]:
            cell_metadata["sample_id"] = []

        # loops through directories to tokenize .{file_type} files
        for file_path in glob.glob(f"{data_directory}/*.{file_type}"):
            print(f"Tokenizing {file_path}")
            file_tokenized_cells, file_cell_metadata = self.tokenize_file(
                file_path, 
                file_type
            )
            tokenized_cells += file_tokenized_cells
            for k in cell_metadata.keys():
                cell_metadata[k] += file_cell_metadata[k]

        return tokenized_cells, cell_metadata

    def tokenize_file(self, file_path, file_type):
        file_cell_metadata = {
            attr_key: [] for attr_key in self.custom_attr_name_dict.keys()
        }
        if file_type == "loom":
            with lp.connect(str(file_path)) as data:
                # define coordinates of detected protein-coding or miRNA genes and vector of their normalization factors
                coding_miRNA_loc = np.where(
                    [self.genelist_dict.get(i, False) for i in data.ra["ensembl_id"]]
                )[0]
                norm_factor_vector = np.array(
                    [
                        self.gene_median_dict[i]
                        for i in data.ra["ensembl_id"][coding_miRNA_loc]
                    ]
                )
                coding_miRNA_ids = data.ra["ensembl_id"][coding_miRNA_loc]
                coding_miRNA_tokens = np.array(
                    [self.gene_token_dict[i] for i in coding_miRNA_ids]
                )

                # define coordinates of cells passing filters for inclusion (e.g. QC)
                try:
                    data.ca["filter_pass"]
                except NameError:
                    var_exists = False
                else:
                    var_exists = True

                if var_exists is True:
                    filter_pass_loc = np.where(
                        [True if i == 1 else False for i in data.ca["filter_pass"]]
                    )[0]
                elif var_exists is False:
                    print(
                        f"{loom_file_path} has no column attribute 'filter_pass'; tokenizing all cells."
                    )
                    filter_pass_loc = np.array([i for i in range(data.shape[1])])

                # scan through .loom files and tokenize cells
                tokenized_cells = []
                for (_ix, _selection, view) in data.scan(items=filter_pass_loc, axis=1):
                    # select subview with protein-coding and miRNA genes
                    subview = view.view[coding_miRNA_loc, :]

                    # normalize by total counts per cell and multiply by 10,000 to allocate bits to precision
                    # and normalize by gene normalization factors
                    subview_norm_array = (
                        subview[:, :]
                        / subview.ca.n_counts
                        * 10_000
                        / norm_factor_vector[:, None]
                    )
                    # tokenize subview gene vectors
                    tokenized_cells += [
                        tokenize_cell(subview_norm_array[:, i], coding_miRNA_tokens)
                        for i in range(subview_norm_array.shape[1])
                    ]

                    # add custom attributes for subview to dict
                    for k in file_cell_metadata.keys():
                        file_cell_metadata[k] += subview.ca[k].tolist()
        elif file_type in ["csv", "tsv"]:
            df = pd.read_csv(file_path, sep="\t" if file_type == "tsv" else ",", index_col=0)
            # Assume genes are columns samples are rows
            df_genes = df.columns
            file_cell_metadata["sample_id"] = df.index.values.tolist()
            coding_miRNA_loc = np.where(
                    [self.genelist_dict.get(i, False) for i in df_genes]
            )[0]
            norm_factor_vector = np.array(
                [
                    self.gene_median_dict[i]
                    for i in df_genes[coding_miRNA_loc]
                ]
            )
            coding_miRNA_ids = df_genes[coding_miRNA_loc]
            coding_miRNA_tokens = np.array(
                [self.gene_token_dict[i] for i in coding_miRNA_ids]
            )
            
            tokenized_cells = []
            for _, row in df.iterrows():
                # select slice with protein-coding and miRNA genes
                _row = row[coding_miRNA_ids]

                # normalize by total counts per cell and multiply by 10,000 to allocate bits to precision
                # and normalize by gene normalization factors
                norm_row = (
                    _row
                    / _row.sum()
                    * 10_000
                    / norm_factor_vector
                )
                # tokenize subview gene vectors
                tokenized_cells.append(
                    tokenize_cell(norm_row.values, coding_miRNA_tokens)
                )

        return tokenized_cells, file_cell_metadata

    def create_dataset(self, tokenized_cells, cell_metadata):
        # create dict for dataset creation
        dataset_dict = {"input_ids": tokenized_cells}
        dataset_dict.update(cell_metadata)

        # create dataset
        output_dataset = Dataset.from_dict(dataset_dict)

        # truncate dataset
        def truncate(example):
            example["input_ids"] = example["input_ids"][0:2048]
            return example

        output_dataset_truncated = output_dataset.map(truncate, num_proc=self.nproc)

        # measure lengths of dataset
        def measure_length(example):
            example["length"] = len(example["input_ids"])
            return example

        output_dataset_truncated_w_length = output_dataset_truncated.map(
            measure_length, num_proc=self.nproc
        )

        return output_dataset_truncated_w_length
    

tokenizer = GenericTranscriptomeTokenizer()
dataset = tokenizer.tokenize_data("depmap", "csv")
dataset

Tokenizing depmap/depmap_expression_ensg_test.csv


Dataset({
    features: ['input_ids', 'sample_id', 'length'],
    num_rows: 5
})

In [3]:
model = BertForMaskedLM.from_pretrained("ctheodoris/Geneformer", 
                                        output_hidden_states=True, 
                                        output_attentions=False)

In [6]:
import torch
def get_likelihoods(samples, model):
    with torch.no_grad():
        output = model(input_ids=samples)
    ls = torch.log_softmax(output.logits, dim=-1)
    B = samples.shape[0]
    N = samples.shape[1]
    likelihoods = [ls[i, range(N), samples[i, :]].sum() for i in range(B)]
    return likelihoods

get_likelihoods(torch.tensor(dataset["input_ids"]), model)
        


[tensor(-6891.1606),
 tensor(-6920.2729),
 tensor(-4699.6240),
 tensor(-5650.2578),
 tensor(-3721.6394)]

In [82]:
t

tensor([[20391, 11958, 20794,  ...,  5117,  2812,  4699],
        [25243, 25363, 25307,  ...,  9731,  2221, 20326],
        [20291, 25243, 25307,  ..., 11046,  3443,   510],
        [24388, 25363, 25243,  ...,  7162,  7280, 13891],
        [20391, 11958, 12705,  ...,  3576,  5671,  1948]])

In [85]:
output.logits[0, range(t.shape[1]), t[0, :]]

tensor([-5.5663, -7.3021,  0.5867,  ...,  9.0273,  6.5536,  6.3689])

In [67]:
torch.log_softmax(output.logits, dim=-1)

tensor([[[-23.1001, -34.5579,  -9.1084,  ..., -17.1373, -16.5247, -22.2942],
         [-23.1712, -35.4161,  -8.6474,  ..., -17.1430, -16.5991, -22.6768],
         [-23.1980, -36.0589,  -8.3384,  ..., -17.2093, -16.3764, -22.7582],
         ...,
         [-23.9251, -37.5456,  -9.2354,  ..., -20.3903, -18.3909, -21.4798],
         [-19.6140, -34.6622,  -9.2514,  ..., -17.2557, -13.3906, -22.0211],
         [-20.7598, -35.3806, -12.2002,  ..., -18.2089, -14.1156, -23.0851]],

        [[-23.1953, -35.7664, -10.2419,  ..., -17.8950, -17.4716, -22.2976],
         [-23.4595, -35.6486,  -9.1513,  ..., -18.4226, -17.3627, -22.3938],
         [-22.9459, -36.2082,  -9.2753,  ..., -18.2061, -17.2145, -22.5077],
         ...,
         [-26.9815, -39.3737, -12.6832,  ..., -24.8926, -21.3464, -24.2824],
         [-27.2365, -37.7310, -15.1323,  ..., -22.2402, -18.4982, -24.5308],
         [-22.2423, -36.0442, -12.2520,  ..., -18.3912, -12.6893, -22.4298]],

        [[-24.7540, -34.4220,  -9.1894,  ...